# Introduction to ForneyLab

ForneyLab is a toolbox for deriving solutions to probabilistic inference problems by message passing on Forney-style factor graphs. A Forney-style factor graph (FFG) is a graphical representation of a factorized probabilistic model. Message passing solutions to inference tasks can be automatically derived from a probabilistic model specification. ForneyLab yields message passing algorithms (MPA) as Julia code. In other words, ForneyLab is a Julia program that automatically builds inference programs that are specified as Julia programs.

These demos assume that the user is familiar with the FFG formalism. We recommend the following introductions: 

1. H.-A. Loeliger, J. Dauwels, J. Hu, S. Korl, Li Ping, and F. Kschischang,
[The factor graph approach to model-based signal processing](https://people.ee.ethz.ch/~papers/docu/aloe-jdau-juhu-skor-2007-1.pdf), Proceedings of the IEEE, vol. 95, no. 6, pp. 1295-1322, June 2007. 
2. Korl, Sascha, [A factor graph approach to signal modelling, system identification and filtering](https://www.research-collection.ethz.ch/handle/20.500.11850/82737), Series in signal and information processing
Doctoral Thesis, 2005

We designed ForneyLab to be practical, while retaining maximal flexibility. The inherent modularity of the FFG framework allowed us to make ForneyLab extensible at all levels (nodes, update rules, algorithms, inference engines). Although we had performance in mind while developing ForneyLab, optimally efficient execution of the resulting inference programs (specified in Julia as message passing sequence) may still require custom work.  

The ForneyLab approach to solving inference problems consists of three phases:

1. **Model specification**. ForneyLab provides a simple meta-language to specifiy models.
2. **Message Passing Agorithm (MPA) Generation**. This task is automatically performed by ForneyLab.
3. **MPA Execution**. This is simply evaluating a Julia program.

Each of the demos will step through these phases in turn, showcasing the most important ForneyLab functionalities. Although the demos each stand on their own, a recommended order would be

1. `introduction`
2. `state_estimation_forward_only`
3. `state_estimation_forward_backward`
4. `bayes_rule_2d`
5. `kalman_smoother_2d`
6. `variational_estimation_iid_gaussian`
7. `variational_estimation_gaussian_mixture`
8. `expectation_propagation`
9. `expectation_maximization`
10. `composite_nodes`
11. `structured_variational_estimation`
12. `hidden_markov_model_estimation`

# Factor Graphs and Variables

A central concept in ForneyLab is the (random) `Variable` type. After including ForneyLab and indicating that we start a new `FactorGraph`, we can declare a `Variable` by calling its constructor function: 

In [1]:
using ForneyLab

# Declare a new graph
g = FactorGraph()

# Declare a variable
x = Variable(id=:x)

g.variables

Dict{Symbol,Variable} with 1 entry:
  :x => Variable(:x, Edges:…

The call to `FactorGraph()` creates a factor graph type and registers the graph as the currently active graph. Note that the variable has been associated with an edge in the currently active graph. 

ForneyLab comes equipped with the `@RV` macro to define random variables. For instance, defining a new variable `y` with identifier `:y` and associating the variable to the current graph can also be accomplished by executing `@RV y`:

In [2]:
@RV y

g.variables

Dict{Symbol,Variable} with 2 entries:
  :y => Variable(:y, Edges:…
  :x => Variable(:x, Edges:…

We can assign a probability distribution to a random variable by the `~` operator:

In [3]:
@RV z ~ GaussianMeanVariance(0.0, 1.0)
g.variables

Dict{Symbol,Variable} with 5 entries:
  :clamp_2 => Variable(:clamp_2, Edges:…
  :y       => Variable(:y, Edges:…
  :clamp_1 => Variable(:clamp_1, Edges:…
  :z       => Variable(:z, Edges:…
  :x       => Variable(:x, Edges:…

Note that the graph now also includes two variables with id `:clamp_1` and `:clamp_2`. These two variables correspond to the mean and variance parameters for the Gaussian and are clamped to values `0.0` and `1.0` respectively.

If you have [graphviz](https://www.graphviz.org/) installed, then you can draw the factor graph. (Edges (variables) that are not constrained by any factor are not drawn):

In [4]:
ForneyLab.draw(g) # draw the graph

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 15950353866949326664 -->
 15950353866949326664 
 
 clamp_1 
 
<!-- 10065977336276825811 -->
 10065977336276825811 
 
 clamp_2 
 
<!-- 1165345475937147423 -->
 1165345475937147423 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 1165345475937147423--15950353866949326664 -->
 1165345475937147423--15950353866949326664 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 1165345475937147423--10065977336276825811 -->
 1165345475937147423--10065977336276825811 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 85377630231327942632 -->
 85377630231327942632 
 
<!-- 85377630231327942632--1165345475937147423 -->
 85377630231327942632--1165345475937147423 
 
 z 
 1 out

In case you don't like the automatically generated id's `:clamp_1` and `:clamp_2`, you could have declared the parameters of the Gaussian distribution through the `@RV` macro and associated a `Clamp` distribution with these variables:

In [5]:
g2 = FactorGraph()
@RV m ~ Clamp(0.0)
@RV v ~ Clamp(1.0)
@RV z ~ GaussianMeanVariance(m, v)
g2.variables

Dict{Symbol,Variable} with 3 entries:
  :m => Variable(:m, Edges:…
  :v => Variable(:v, Edges:…
  :z => Variable(:z, Edges:…

In [6]:
ForneyLab.draw(g2)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 11213783859701046526 -->
 11213783859701046526 
 
 clamp_1 
 
<!-- 12029872037016350493 -->
 12029872037016350493 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 12029872037016350493--11213783859701046526 -->
 12029872037016350493--11213783859701046526 
 
 m 
 1 out 
 2 m 
 
<!-- 17616024178248183665 -->
 17616024178248183665 
 
 clamp_2 
 
<!-- 12029872037016350493--17616024178248183665 -->
 12029872037016350493--17616024178248183665 
 
 v 
 1 out 
 3 v 
 
<!-- 119702704987966121122 -->
 119702704987966121122 
 
<!-- 119702704987966121122--12029872037016350493 -->
 119702704987966121122--12029872037016350493 
 
 z 
 1 out

The graph stores the identifier of each variable. This is useful because now we can retrieve a variable from a graph by its identifier, .e.g.,

In [7]:
g2.variables[:m]

Variable(:m, Edges:
Edge belonging to variable m: ( clamp_1.i[out] )----( gaussianmeanvariance_1.i[m] ).
)

Let's build another simple factor graph for 
$$\begin{align*}
p(x,y,z) &= p(z|x,y)\,p(x)\,p(y) \\
  &= \delta(z-x-y)\,\mathcal{N}(x\,|\,0.0,1.0)\,\mathcal{N}(y\,|\,2.0,3.0)
\end{align*}$$

In [8]:
g3 = FactorGraph()
@RV x ~ GaussianMeanVariance(0.0, 1.0)
@RV y ~ GaussianMeanVariance(2.0, 3.0)
@RV z = x + y
ForneyLab.draw(g3)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 6250434450868393432 -->
 6250434450868393432 
 
 clamp_3 
 
<!-- 4027082208151477709 -->
 4027082208151477709 
 
 clamp_2 
 
<!-- 15352174231597294789 -->
 15352174231597294789 
 
 + 
 addition_1 
 
<!-- 14216082688881268699 -->
 14216082688881268699 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 15352174231597294789--14216082688881268699 -->
 15352174231597294789--14216082688881268699 
 
 x 
 1 out 
 2 in1 
 
<!-- 17575400511439571852 -->
 17575400511439571852 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 15352174231597294789--17575400511439571852 -->
 15352174231597294789--17575400511439571852 
 
 y 
 1 out 
 3 in2 
 
<!-- 14216082688881268699--4027082208151477709 -->
 14216082688881268699--4027082208151477709 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 732712404331934010 -->
 732712404331934010 
 
 clamp_1 
 
<!-- 14216082688881268699--732712404331934010 -->
 14216082688881268699--732712404331934010 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 7240580548003578834 -->
 7240580548003578834 
 
 clamp_4 
 
<!-- 17575400511439571852--6250434450868393432 -->
 17575400511439571852--6250434450868393432 
 
 clamp_3 
 1 out 
 2 m 
 
<!-- 17575400511439571852--7240580548003578834 -->
 17575400511439571852--7240580548003578834 
 
 clamp_4 
 1 out 
 3 v 
 
<!-- 104498375777127387252 -->
 104498375777127387252 
 
<!-- 104498375777127387252--15352174231597294789 -->
 104498375777127387252--15352174231597294789 
 
 z 
 1 out

Next, we could be interested in inferring a property of `z`, e.g., the mean parameter for the distribution over `z`. This process can be automated by message passing in the graph. The next set of demo's will expand on the various methods for describing graphs and message passing inference methods with ForneyLab.